In [2]:
import pyaudio
import numpy as np
import librosa
import threading
import keyboard

# Global variables
running = True
pitch_shift_factor = 2

def pitch_shift(signal, factor):
    # Normalize the input signal to the range [-1, 1]
    signal = signal.astype(np.float32) / np.max(np.abs(signal))

    # Use librosa for pitch shifting
    shifted_signal = librosa.effects.pitch_shift(signal, sr=44100, n_steps=factor)

    return shifted_signal

def stop_program():
    global running
    keyboard.wait("q")
    running = False

def adjust_pitch_factor():
    global pitch_shift_factor

    while running:
        if keyboard.is_pressed("+"):
            pitch_shift_factor += 1
            print(f"Pitch shift factor increased to {pitch_shift_factor}")
            keyboard.wait("release +")  # Wait for key release to prevent rapid changes
        elif keyboard.is_pressed("-"):
            pitch_shift_factor -= 1
            print(f"Pitch shift factor decreased to {pitch_shift_factor}")
            keyboard.wait("release -")  # Wait for key release to prevent rapid changes

def main():
    # Parameters
    chunk_size = 1024  # Number of frames per buffer
    format = pyaudio.paInt16
    channels = 1
    rate = 44100  # Sampling rate

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open stream
    stream = p.open(format=format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    output=True,
                    frames_per_buffer=chunk_size)

    print("Real-time pitch shift. Press 'q' to stop.")
    print("Press '+' to increase pitch shift factor, '-' to decrease.")

    # Start threads for program termination and pitch factor adjustment
    stop_thread = threading.Thread(target=stop_program)
    pitch_adjust_thread = threading.Thread(target=adjust_pitch_factor)

    stop_thread.start()
    pitch_adjust_thread.start()

    try:
        while running:
            # Read audio input from the microphone
            input_data = stream.read(chunk_size)
            input_array = np.frombuffer(input_data, dtype=np.int16)

            # Perform pitch shift
            shifted_array = pitch_shift(input_array, pitch_shift_factor)

            # Convert the array back to bytes
            output_data = (shifted_array * 32767).astype(np.int16).tobytes()

            # Play the shifted audio
            stream.write(output_data)

    except KeyboardInterrupt:
        pass

    finally:
        # Stop the threads
        stop_thread.join()
        pitch_adjust_thread.join()

        # Close the stream and PyAudio
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    main()


Real-time pitch shift. Press 'q' to stop.
Press '+' to increase pitch shift factor, '-' to decrease.


Exception in thread Thread-9 (adjust_pitch_factor):
Traceback (most recent call last):
  File "D:\anaconda\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\anaconda\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\viswa\AppData\Local\Temp\ipykernel_47768\249407621.py", line 32, in adjust_pitch_factor
  File "D:\anaconda\Lib\site-packages\keyboard\__init__.py", line 881, in wait
    remove = add_hotkey(hotkey, lambda: lock.set(), suppress=suppress, trigger_on_release=trigger_on_release)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\keyboard\__init__.py", line 641, in add_hotkey
    steps = parse_hotkey_combinations(hotkey)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\keyboard\__init__.py", line 573, in parse_hotkey_combinations
    return tuple(tuple(combine_step(step)) fo

Pitch shift factor increased to 3
